In [ ]:
!pip install scipy
!pip install xformers
!pip install numpy==1.23.3

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch
import transformers
import shap
import xformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
test_data = pd.read_csv("data/test_new.csv")

# Converting bools to binary
test_data["labels"] = test_data["labels"].astype(int)

idx = 43 # Which article to predict and explain
df = test_data.head(10)
x = [test_data["text"][idx]]

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('chcaa/dfm-encoder-large-v1', do_lower_case=True)
tokenizer.model_max_length = 512
tokenizer.max_length = 512
tokenizer.truncation = True

In [ ]:
model = BertForSequenceClassification.from_pretrained('model_test_new1', num_labels=2)

pipe = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None, padding=True, truncation=True, max_length = 512) # , padding=True, truncation=True

pred = pipe(x, padding=True, truncation=True, max_length = 512)

In [ ]:
test_data['predictions'] = None # add a new column for predictions

# appending predictions to the predictions column
for preds in range(len(pred)):
    if pred[preds][0]['label'] == 'LABEL_0':
        test_data.loc[preds, 'predictions'] = 0
    else:
        test_data.loc[preds, 'predictions'] = 1

In [ ]:
# How many articles are correctly and wrongly classified 
(test_data['labels'] == test_data['predictions']).value_counts()

In [ ]:
print(pred)

In [ ]:
# Create an explainer for the pipeline
explainer = shap.Explainer(pipe)

# Compute SHAP values
shap_values = explainer(x)

# Create SHAP plot
shap.plots.text(shap_values)